In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import pandas as pd
import os
import random
from random import choice
import geocoder

import requests
from fake_useragent import UserAgent

In [2]:
dat = pd.read_csv('attraction_reviews.csv')

In [3]:
dat.tail(5)

,image_user,name_user,info_user,type_review,date_review,text_review,score_user,attraction_id
12694,https://dynamic-media-cdn.tripadvisor.com/medi...,nypuck,"Santa Fe, NM85 contributions",Come see where the Republicans live in NYC.,"September 5, 2012",Walk the streets to see the upper class and th...,3.0,100
12695,https://dynamic-media-cdn.tripadvisor.com/medi...,Carol K,"New York City, NY440 contributions",Nice for a walk,"August 17, 2012",The houses along 5th Ave include some handsome...,3.0,100
12696,https://dynamic-media-cdn.tripadvisor.com/medi...,Pili_12,"New York City, NY24 contributions",Nice to walk,"June 18, 2012",although it may be too expensive to stop to ea...,5.0,100
12697,https://dynamic-media-cdn.tripadvisor.com/medi...,LazyPoint,"New York City, NY235 contributions",I live here,"June 11, 2012",Great museums and a few great restaurants (Caf...,5.0,100
12698,https://dynamic-media-cdn.tripadvisor.com/medi...,yourtravelchick,"Ridgewood, NJ134 contributions",It's not just all Madison Ave!,"March 13, 2012",I like to wander up through Central Park and t...,5.0,100


In [4]:
dat.shape

(12699, 8)

In [5]:
dat.isnull().sum()

image_user       1167
name_user           0
info_user           0
type_review         0
date_review         0
text_review         0
score_user          0
attraction_id       0
dtype: int64

In [4]:
data = pd.read_csv('attractions_data1.csv')

In [5]:
data.shape

(150, 15)

In [8]:
data.isnull().sum()

attration_id    0
country         0
province        0
city            0
lien_detail     0
image           0
titre           0
nbre_review     0
address         0
description     0
score           0
price           0
dtype: int64

In [9]:
data.tail(5)

,attration_id,country,province,city,lien_detail,image,titre,nbre_review,address,description,score,price
145,145,United States,New York (NY),New York City,https://www.tripadvisor.com/Attraction_Review-...,"['data:image/svg+xml;utf8,%3C%3Fxml%20version%...",146. The Metropolitan Opera,3106,"{'lat': 40.77602386, 'lng': -73.98724365}",Operas • Theater & Performances,5.0,100.0
146,146,United States,New York (NY),New York City,https://www.tripadvisor.com/Attraction_Review-...,"['data:image/svg+xml;utf8,%3C%3Fxml%20version%...",147. The Magician New York,524,"{'lat': 40.74420166, 'lng': -73.98829651}",Theater & Performances,5.0,-1.0
147,147,United States,New York (NY),New York City,https://www.tripadvisor.com/Attraction_Review-...,"['data:image/svg+xml;utf8,%3C%3Fxml%20version%...",148. The Phantom of the Opera,6086,"{'lat': 40.75992584, 'lng': -73.98041534}",Operas • Theater & Performances,4.5,-1.0
148,148,United States,New York (NY),New York City,https://www.tripadvisor.com/Attraction_Review-...,"['data:image/svg+xml;utf8,%3C%3Fxml%20version%...",149. Williamsburg Bridge,193,"{'lat': 40.70780182, 'lng': -74.01190186}",Bridges,4.5,-1.0
149,149,United States,New York (NY),New York City,https://www.tripadvisor.com/Attraction_Review-...,"['data:image/svg+xml;utf8,%3C%3Fxml%20version%...",150. Pip's Island,80,"{'lat': 40.75611115, 'lng': -74.00055695}",Theaters,5.0,-1.0


In [12]:
#get = data[['attration_id', 'lien_detail']][101:120] je m etais arrete a 100 la fois passé 

In [9]:
get = data[['attration_id', 'lien_detail']][101:105]    # ici jai continué a 101

In [10]:
get['lien_detail'][101]

'https://www.tripadvisor.com/AttractionProductReview-g60763-d24962419-City_Climb_The_Ultimate_Skyscraping_Adventure_at_Edge-New_York_City_New_York.html'

In [11]:
get

,attration_id,lien_detail
101,101,https://www.tripadvisor.com/AttractionProductR...
102,102,https://www.tripadvisor.com/AttractionProductR...
103,103,https://www.tripadvisor.com/AttractionProductR...
104,104,https://www.tripadvisor.com/AttractionProductR...


In [14]:
ligne = get.shape[0]
ligne

19

In [12]:
def clean_text(text):
    text = text.strip().strip('-')
    text = text.lower()
    text = '_'.join(text.split(' '))
    return text

In [13]:
def reform(url):
    lien = url.split('Reviews')

    return lien[0], lien[1]

In [14]:
def getItem(item):
    return item.getText().strip() if item is not None else None
 


In [15]:
def scrapAllReviewsGet(url, attraction_id):
    
    items = []
    user_agent = UserAgent()
    userAgent = user_agent.random
    Firefox_options = webdriver.EdgeOptions()
    Firefox_options.add_argument(f'user-agent={userAgent}')
    driver = webdriver.Edge(options=Firefox_options)
    driver.get(url)
    #driver.set_window_size(1080, 800)
    time.sleep(random.uniform(5,17))
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    #card_reviews = soup.find('div', {"class", 'LbPSX'})
    #reviews = card_reviews.find_all('div', {"class", '_c'})

    reviews = soup.select('div[data-automation="tab"] div.C div._c') #div_country.find_all('div', attrs={"class":"KCGqk _T Cj"}) 

    for review in reviews:
        item = {}
        photo_user = review.find('div', {"class", 'MLvbw f u'}) #
        photo_user = photo_user.find('a', {'class', 'BMQDV _F Gv wSSLS SwZTJ'})

       

        try:
            photo_user = photo_user.find('img')
            item['image_user'] = photo_user['src'] 
        except:
            item['image_user'] = ''
             

        
        user = review.find('div', {"class", 'zpDvc Zb'})
        item['name_user'] = getItem(user.find('span', {'class', 'biGQs _P fiohW fOtGX'}))
        item['info_user'] = getItem(user.find('div', {'class', 'JINyA'}))
        item['type_review'] = getItem(review.find('span', {'class': 'yCeTE'}))
        item['date_review'] = getItem(review.find('div', {'class': 'biGQs _P pZUbB ncFvv osNWb'})).split(' ', 1)[1]
        item['text_review'] = getItem(review.find('div', {'class': 'fIrGe _T bgMZj'}))
        span_score = review.find('svg', {'class': 'UctUV d H0'})  #UctUV d H0
        item['score_user'] = float(clean_text(span_score.get('aria-label')).replace('_of_5_bubbles',''))
        item['attraction_id'] = attraction_id

        #print(item)


        items.append(item)
    # appuye sur le bouton suivant et relancer la fonction pour scraper
    
    
    print(f'nombre data scrapé: ', {len(items)})
    driver.quit()

    return items, len(items)




In [39]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import pandas as pd
import os
import random
from random import choice
import geocoder

import requests
from fake_useragent import UserAgent


def scrapAllReviewsGet__(url, attraction_id):
    
    items = []
    user_agent = UserAgent()
    userAgent = user_agent.random
    Firefox_options = webdriver.EdgeOptions()
    Firefox_options.add_argument(f'user-agent={userAgent}')
    driver = webdriver.Edge(options=Firefox_options)
    driver.get(url)
    #driver.set_window_size(1080, 800)
    time.sleep(random.uniform(5,17))
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    reviews = soup.find_all("span", {"class", 'QewHA H4 _a'}) #div_country.find_all('div', attrs={"class":"KCGqk _T Cj"}) 

    for review in reviews:
        print(review)
        
    driver.quit()

    return items, len(reviews)

In [40]:
scrapAllReviewsGet__("https://www.tripadvisor.com/Hotel_Review-g60763-d6385164-Reviews-CitizenM_New_York_Times_Square-New_York_City_New_York.html#REVIEWS", 1)

<span class="QewHA H4 _a" data-test-target="review-text"><span>For our first visit in New York we choose this location for its position, design and services. We are very satisfy after staying there and all our impression before the booking are respected. The room has a perfect layout compared to its size, and the bed is big and very comfortable. Rooftop bar is amazing. We don't see the gym and we not taste breakfast neither canteen. All the staff is very kind and helpful.</span></span>
<span class="QewHA H4 _a" data-test-target="review-text"><span>Excellent central location for sight seeing. Was also very quiet and no issues with noise despite being so central. The rooms are very small and cramped for 2 people, the sink is outside the bathroom and the cabin style bathroom is awkward if you aren't used to sharing! However I raised my concerns to the concierge staff who were so helpful and accommodating and we were able to move to a bigger room, which had a fully enclosed  separate bathr

([], 10)

In [21]:
scrapAllReviewsGet("https://www.tripadvisor.com/AttractionProductReview-g60763-d16822065-or10-Private_New_York_City_Airport_Arrival_Transfer_by_Luxury_Stretch_Limo-New_York.html", 0)

nombre data scrapé:  {5}


([{'image_user': 'https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1a/f6/ee/10/default-avatar-2020-8.jpg?w=100&h=-1&s=1',
   'name_user': 'cakeene',
   'info_user': 'Columbia, KY1 contribution',
   'type_review': 'Excellent service',
   'date_review': 'July 10, 2020',
   'text_review': 'It was amazing, it felt like, and I was treated like a rockstar. Everything was exactly how you imagine it, and service was top notch. It really set the tone for the whole trip.',
   'score_user': 5.0,
   'attraction_id': 0},
  {'image_user': 'https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1a/f6/e2/4a/default-avatar-2020-42.jpg?w=100&h=-1&s=1',
   'name_user': 'Kristy K',
   'info_user': '1 contribution',
   'type_review': 'Great service!',
   'date_review': 'July 10, 2020',
   'text_review': 'Clean and friendly. Very impressed by the high quality of service. Would highly recommend. Overall a lovely experience.',
   'score_user': 5.0,
   'attraction_id': 0},
  {'image_user': 'https://dyn

In [22]:
def createFileCoppyData(items, chemin_fichier_csv = 'data.csv'):
    # cette fonction va prendre le tableau de dictionnaire et
    # enregistrer chaque ligne dans le tableau

    items_data = pd.DataFrame(items)
    items_data.to_csv(chemin_fichier_csv, mode='a', header=not os.path.exists(chemin_fichier_csv),  index=False)


In [23]:
def iterationsReviews(data):
    for index, row in data.iterrows():
        print(index, "---scraping", row['lien_detail'], " attraction id:", row['attration_id'])
        arret = 0
        for i in range(0, 25):
            iter = i * 10                                                                           # 00 01 02 03 04 05 06
            print(f'itteration: ', {i})
            time.sleep(random.uniform(5,8))
            url = reform(row['lien_detail'])[0] + "Reviews-or" + str(iter) + reform(row['lien_detail'])[1]
            print(url)

            items, len_get = scrapAllReviewsGet(url, row['attration_id']) 
            createFileCoppyData(items, 'attraction_reviews.csv')
            arret += len_get
            print("total scrapé: ", arret)
            if arret >= 140:
                print(index, '--- fin ---')
                break
            

        



In [ ]:
https://www.tripadvisor.com/Attraction_Review-g60763-d105125-Reviews-or10-The_Metropolitan_Museum_of_Art-New_York_City_New_York.html

In [34]:
iterationsReviews(get)

100 ---scraping https://www.tripadvisor.com/Attraction_Review-g60763-d136033-Reviews-Upper_East_Side-New_York_City_New_York.html  attraction id: 100
itteration:  {0}
https://www.tripadvisor.com/Attraction_Review-g60763-d136033-Reviews-or0-Upper_East_Side-New_York_City_New_York.html
nombre data scrapé:  {10}
total scrapé:  10
itteration:  {1}
https://www.tripadvisor.com/Attraction_Review-g60763-d136033-Reviews-or10-Upper_East_Side-New_York_City_New_York.html
nombre data scrapé:  {10}
total scrapé:  20
itteration:  {2}
https://www.tripadvisor.com/Attraction_Review-g60763-d136033-Reviews-or20-Upper_East_Side-New_York_City_New_York.html
nombre data scrapé:  {10}
total scrapé:  30
itteration:  {3}
https://www.tripadvisor.com/Attraction_Review-g60763-d136033-Reviews-or30-Upper_East_Side-New_York_City_New_York.html
nombre data scrapé:  {10}
total scrapé:  40
itteration:  {4}
https://www.tripadvisor.com/Attraction_Review-g60763-d136033-Reviews-or40-Upper_East_Side-New_York_City_New_York.html
n

KeyboardInterrupt: 

In [25]:
a, b = reform("https://www.tripadvisor.com/Hotel_Review-g60763-d6385164-Reviews-CitizenM_New_York_Times_Square-New_York_City_New_York.html#REVIEWS")

In [26]:
a+"Reviews-or"+b

'https://www.tripadvisor.com/Hotel_Review-g60763-d6385164-Reviews-or-CitizenM_New_York_Times_Square-New_York_City_New_York.html#REVIEWS'

In [ ]:
b

In [ ]:
reform("https://www.tripadvisor.com/Attraction_Review-g60763-d102741-Reviews-Brooklyn_Bridge-New_York_City_New_York.html")[0]